We start by importing the necessary libraries and setting up the paths for the files. But before doing anything, we need the EEG data to work with. Use the `ds003846-1sh` script in this repo to download the data (it will be saved in the`ds003846-1.0.0` folder). The data is organized into folders by subject, and each folder has sessions for different conditions. We’ll be doing our analysis on subject 1 (sub-1). 


In [14]:
import mne
import pandas as pd
from pathlib import Path
import numpy as np
import os

base_path = Path("./ds003846-2.0.2")
subject = "sub-02"
session = "ses-EMS"
data_path = base_path / subject / session / "eeg"

# define paths
base_path = Path("./ds003846-2.0.2")
subject = "sub-02"
data_paths = {
    "ses-EMS": base_path / subject / "ses-EMS" / "eeg",
    "ses-Visual": base_path / subject / "ses-Visual" / "eeg",
    "ses-Vibro": base_path / subject / "ses-Vibro" / "eeg"
}



 **Load the data (Step 1)**  

First, we’ll load EEG data for all sessions (EMS, Visual, and Vibro) and apply 10-20 montage. The dataset uses channel names like `BrainVision RDA_Fp1`, which won’t match MNE’s standard montages. We’ll strip the prefix and apply the 10-20 montage.

In [ ]:
# We load the EEG Data for subject 1 for all sessions/conditions
raw_sessions = {}
for session, path in data_paths.items():
    print(f"Loading data for {session}...")
    vhdr_file = path / f"{subject}_{session}_task-PredictionError_eeg.vhdr"
    raw = mne.io.read_raw_brainvision(str(vhdr_file), preload=True)
    raw_sessions[session] = raw
    # print some metadata
    print(f"{session}: Channels={raw.info['nchan']}, Sampling rate={raw.info['sfreq']} Hz, Samples={raw.n_times}, Duration={raw.times[-1]:.2f} s")
    print(f"Original channels for {session}: {raw.info['ch_names']}")


# The channel names in the dataset have a prefix ("BrainVision RDA_") that is specific to the recording system
# We need to remove this prefix to match the standard EEG channel naming convention (e.g., "Fp1", "Fz")
for session, raw in raw_sessions.items():
    print(f"Cleaning channel names for {session}...")
    raw.rename_channels(lambda x: x.replace("BrainVision RDA_", ""))
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage)
    raw.plot_sensors(kind="topomap", show_names=True)

**Filter and downsample (Step 2)**  

We start the preporcessing --> filter and downsample the data. The paper says: “The raw EEG data was re-sampled to 250 Hz, high-pass filtered at 1 Hz, and low-pass filtered at 125 Hz.” However, filtering should happen before downsampling to avoid aliasing, so we’re doing it in that order instead. Their MATLAB does the same, so no difference here.

In [ ]:
for session, raw in raw_sessions.items():
    # let's first check the frequency range of the data before bandpass
    psd, freqs = raw.compute_psd().get_data(return_freqs=True)
    max_freq = freqs.max()
    min_freq = freqs.min()
    print(f"Frequency range in the data: {min_freq} Hz to {max_freq} Hz")
    # Filter the Data (1–125 Hz bandpass)
    print(f"Filtering and downsampling for {session}...")
    raw.filter(l_freq=1, h_freq=124.9)  # Bandpass filter (1-125 Hz)
    raw.resample(250)  # Downsample to 250 Hz

**Re-referencing (Step 3)**  
In the paper, they re-reference the data to the average of all electrodes, including FCz. We’ll follow that here too.

In [ ]:
for session, raw in raw_sessions.items():
    print(f"Re-referencing data for {session}...")
    raw.set_eeg_reference("average")

In [ ]:
# **Epoch Rejection **
# The paper divides data into 1-second epochs and removes the noisiest 15%. This is skipped in our implementation to retain continuous data for ICA, simplifying the pipeline.
# The paper mentions rejecting 15% of the noisiest 1-second epochs before ICA to remove extreme artifacts. However, for simplicity, we skip this step.
# Skipping this step retains more data for analysis, but we acknowledge it might slightly reduce the quality of ICA results.


# for session, raw in raw_sessions.items():
#     print(f"Rejecting 15% of noisiest epochs for {session}...")
#     # Use autoreject or similar methods here if needed
#     pass

# %% [markdown]


**ICA analysis (step 4)**

Now we perform ICA analysis to remove artifacts like eye blinks and line noise.
The paper also does this but after their intial 1 second window epoching and noisy epochs removal. We think this might be because it helps the ICA result when the data is clean. But in our pipeline we directly do the ICA analysis on the raw continous data.

In [ ]:

# %%
ica_sessions = {}
for session, raw in raw_sessions.items():
    print(f"Running ICA for {session}...")
    ica = mne.preprocessing.ICA(n_components=20, random_state=42, max_iter="auto")
    ica.fit(raw)
    ica.plot_components()  # Inspect components manually

    # TO DO: we need to identify the bad components by visually inspecting, not sure how to interpret the plots
    ica.exclude = []  # Adjust based on visual inspection
    ica.apply(raw)
    ica_sessions[session] = raw

**ERP-Specific Filtering (Step 5)**

To isolate ERP-related frequencies, the paper specifies filtering the data at 0.2–35 Hz. This helps focus on the relevant signals
and is applied to the ICA-cleaned data.

In [ ]:
for session, raw in ica_sessions.items():
    print(f"Filtering for ERP analysis in {session}...")
    raw.filter(l_freq=0.2, h_freq=35)

**Epoching around stimulus (step 6)**

We create epochs aligned to the stimulus event `box:touched` which is the moment of object selection. In the paper they state "Then we sliced it between -0.3 seconds to 0.7 seconds around the stimulus onset, i.e. the moment of object selection". We think the `box:touched` probably correponds to the "moment of object selection" so thats what we are using to align out epochs. 

In [ ]:
def classify_event()
    event_id = {"box:touched": 2}


In [ ]:
epochs_sessions = {}
tmin, tmax = -0.3, 0.7
event_id = {"normal": 2, "conflict": 3}

for session, raw in ica_sessions.items():
    print(f"Creating stimulus-aligned epochs for {session}...")
    # Extract events
    events_file = data_paths[session] / f"{subject}_{session}_task-PredictionError_events.tsv"
    events_df = pd.read_csv(events_file, sep="\t")
    print(events_df.head(3))

    # Map events
    events = []
    for _, row in events_df.iterrows():
        event_dict = dict(item.split(":") for item in row["value"].split(";"))

        if "box" in event_dict and event_dict["box"] == "touched":
            sample = int(row["sample"])

            events.append([sample, 0, event_id[event_dict["normal_or_conflict"]]])

    events = np.array(events)
    # Epoching, drop duplicate event too
    epochs = mne.Epochs(
        raw, events, event_id=event_id, tmin=tmin, tmax=tmax,
        baseline=(None, 0), preload=True, event_repeated="drop"
    )
    epochs_sessions[session] = epochs
    print(f"Extracted {len(epochs)} epochs for {session}.")

**Epoch Rejection (Step 7)**

Following the paper, we reject the 10% noisiest epochs. 

In [ ]:
for session, epochs in epochs_sessions.items():
    print(f"Rejecting 10% of noisiest epochs for {session}...")
    reject_criteria = dict(eeg=200e-6)  # need to find a good threshold
    epochs.drop_bad(reject=reject_criteria)

**ERP Analysis (step 8)**

In this step, we calculate and visualize the ERP by averaging the epochs. However, unlike the paper, we are not separating the epochs into matching and non-matching trials because the dataset does not include specific markers for this. A possible workaround could involve analyzing the order of events (e.g., box:touched, visual:off, etc.), but this would require additional assumptions.

Also, the paper mentions focusing on the FCz channel for ERP analysis, but this channel is not present in the dataset for participant 1. Therefore, we use the Fz channel as an alternative for the analysis.

In [ ]:
epochs_sessions

In [ ]:

for session, epochs in epochs_sessions.items():
    print(f"Computing ERPs for {session}...")

    # Debugging Step: Check channel names
    print(f"Channels available in {session}: {epochs.info['ch_names']}")

    # Use FCz if available, otherwise default to Fz
    print(epochs.info["ch_names"])
    channel_to_use = "FCz" if "FCz" in epochs.info["ch_names"] else "Fz"

    if channel_to_use not in epochs.info["ch_names"]:
        print(f"Warning: Neither FCz nor Fz found in {session}. Skipping this session.")
        continue  # Skip the session if neither FCz nor Fz is available

    print(f"Using {channel_to_use} for ERP computation in {session}.")
    # evoked = epochs["box:touched"].average(picks=channel_to_use)
    evoked = epochs["normal"].average(picks=channel_to_use)
    # Plot the ERP
    fig = evoked.plot()
    fig.suptitle(f"ERP for box:touched ({channel_to_use}) in {session}")  # Add a descriptive title

In [70]:
import matplotlib.pyplot as plt

In [ ]:

for session, epochs in epochs_sessions.items():
    print(f"Computing ERPs for {session}...")

    # Debugging Step: Check channel names
    print(f"Channels available in {session}: {epochs.info['ch_names']}")

    # Use FCz if available, otherwise default to Fz
    print(epochs.info["ch_names"])
    channel_to_use = "FCz" if "FCz" in epochs.info["ch_names"] else "Fz"

    if channel_to_use not in epochs.info["ch_names"]:
        print(f"Warning: Neither FCz nor Fz found in {session}. Skipping this session.")
        continue  # Skip the session if neither FCz nor Fz is available

    print(f"Using {channel_to_use} for ERP computation in {session}.")
    # # evoked = epochs["box:touched"].average(picks=channel_to_use)
    # evoked = epochs["conflict"].average(picks=channel_to_use)
    # fig = evoked.plot()
    # fig.suptitle(f"ERP for conflict ({channel_to_use}) in {session}")  # Add a descriptive title

    # Compute evoked responses for both conditions
    evoked_conflict = epochs["conflict"].average(picks=channel_to_use)
    evoked_normal = epochs["normal"].average(picks=channel_to_use)

    # Create a new figure for plotting
    plt.figure(figsize=(10, 8))

    # Plot for conflict
    evoked_conflict.plot(titles=f"ERP for conflict ({channel_to_use}) in {session}")

    # Plot for normal in a new figure
    plt.figure(figsize=(10, 8))
    evoked_normal.plot(titles=f"ERP for normal ({channel_to_use}) in {session}")

    plt.show() 